In [1]:
from sys import platform
import os
import cv2
import json
from pupil_apriltags import Detector
import open3d as o3d
from glob import glob
import numpy as np
from scipy.spatial.transform import Rotation as R
import collections
import k3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
from math import pi
def position_error(T_e, T_gt):

    return np.linalg.norm(T_e- T_gt)


def rotation_error(R_e, R_gt):
    
    return (180/pi) * np.arccos((np.trace(np.matmul(R_e, R_gt.T))-1)/2)

In [58]:
def k3d_frustrum(pose, name, size=0.009, color=0x0000ff):
    # i.e. not using image sizes 
    pos = pose[:3, 3]
    
    forward = -pose[:3, 2] * size * -1.4
    right = -pose[:3, 0] * size * 1.25
    up = -pose[:3, 1] * size
    
    #verts = [pos, pos + forward*size ]
    verts = [pos, pos + forward - right*0.5 + up*0.5, pos + forward + right * 0.5 + up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 - up*0.5, pos + forward + right * 0.5 - up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 + up*0.5, pos + forward - right * 0.5 - up * 0.5, pos]
    verts += [pos, pos + forward + right*0.5 + up*0.5, pos + forward + right * 0.5 - up * 0.5, pos]
    return k3d.line(verts, color=color, width=2.5, name = name, shader="simple")

def visualization_registration(reg, mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffcc00, 0xccff00, 0x00ccff]

    for i, frame in enumerate(reg.keys()):
        pos = reg[frame][:3, 3] 
        vertices += pos.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame), size=0.1, color=colors[i % len(colors)]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()

at_detector = Detector(
            families="tagStandard41h12",
            nthreads=1,
            quad_decimate=1.0,
            quad_sigma=0.0,
            refine_edges=1,
            decode_sharpening=0.25,
            debug=0,
        )


Register = collections.namedtuple(
    "RegisterInfo", ["CameraPose", "Intrinsic", "TagPose"])


In [50]:
evaluation

{'3': {'Real_Distance': [2.7917097018357655,
   2.7973448606416436,
   2.796578449915016,
   2.6850541169356816,
   2.41190888133702,
   2.290969355263533,
   2.145363393180526,
   2.2352307574050116,
   5.805187070598251,
   5.701245185380615,
   5.451021804579334],
  'Frame_id': ['frame_00000.jpg',
   'frame_00006.jpg',
   'frame_00013.jpg',
   'frame_00019.jpg',
   'frame_00026.jpg',
   'frame_00033.jpg',
   'frame_00039.jpg',
   'frame_00045.jpg',
   'frame_00770.jpg',
   'frame_00777.jpg',
   'frame_00788.jpg'],
  'Error_Stability': [2.257453483404485e-15,
   1.3692750637043598e-15,
   1.96139401017111e-15,
   2.4424906541753444e-15,
   1.7393494052460785e-15,
   1.9984014443252818e-15,
   5.921189464667501e-16,
   4.070817756958907e-16,
   8.733754460384565e-15,
   8.585724723767877e-15,
   2.6645352591003757e-15],
  'Error_Accuracy': [2.282408062429674,
   2.2195212150608015,
   2.11006176586676,
   1.6986748145871218,
   0.753793827005417,
   0.43836024770560655,
   0.125477873

In [46]:
if platform == "linux" or platform == "linux2":  
# linux
    data_path  = "/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1"
    data_pi_path = "/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1"
    evaluation_json_path = r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/evaluation_apriltag_detection_Iphone.json"
elif platform == "win32":
# Windows...
    data_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1"
    data_pi_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1"
    evaluation_json_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\evaluation_apriltag_detection_Iphone_a1d1.json"

with open(evaluation_json_path, "r") as f:
    evaluation  = json.load(f)

r = R.from_euler('xyz', [0, 180, -90], degrees=True)
Additional_Rotation = r.as_matrix()
additional_rotation = np.concatenate(
                     [np.concatenate([Additional_Rotation, np.array([[0], [0], [0]])], axis=1), np.array([[0, 0, 0, 1]])], axis=0)

# Select one frame from iphone for each tag as register
track_frame = {}
for tag_id in evaluation.keys():
    distance_error = [alpha_error * 0.087 for alpha_error in evaluation[tag_id]["Error_Accuracy"]]
    distance_error = np.array(distance_error)

    index = np.argmin(distance_error)
    frame_id = evaluation[tag_id]["Frame_id"][index]

    track_frame[tag_id] = frame_id

iphone_frames_folder= os.path.join(data_path, "frames")
iphone_pose_folder = os.path.join(data_path, "pose")
mesh_fullpath = os.path.join(data_path, "data3d/textured_output.obj")

# Extract the camera pose and tag pose of each register
register = {}

for tag_id, frame_id in track_frame.items():

    camera_param_file = frame_id.replace(".jpg", ".json")
    img = cv2.imread(os.path.join(iphone_frames_folder, frame_id), cv2.IMREAD_GRAYSCALE)

    img_height, img_width = img.shape
    with open(os.path.join(iphone_pose_folder, camera_param_file), 'r') as f:
        camera_param = json.load(f)

    intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)
    Cam2World = np.array(camera_param["cameraPoseARFrame"]).reshape(4, 4)

    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=0.087)

    for tag in tags:
        if tag.tag_id == int(tag_id):
            tag_pose = np.concatenate(
                    [np.concatenate([np.array(tag.pose_R), np.array(tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
            register[tag.tag_id] = Register(CameraPose=Cam2World, Intrinsic=intrinsics, TagPose=tag_pose)


# Compute the Registration 
# 1. Detect the tag in the frame
# 2. Find the register with common tag
# 3. Transform the new frame to the world coordinate

PI_registration = {}
PI_images_folder = os.path.join(data_pi_path, "images_undistorted")
PI_intrinsics = np.array([[766.2927454396544, 0.0, 543.6272327745995],
                            [0.0, 766.3976103393867, 566.0580149497666],
                            [0.0, 0.0, 1.0]])
PI_fxfycxcy= [PI_intrinsics[0, 0], PI_intrinsics[1, 1], PI_intrinsics[0, 2], PI_intrinsics[1, 2]]
images = os.listdir(PI_images_folder)
images.sort()

for i, img_id in enumerate(images):
    if i % 20 != 0:
        continue

    img = cv2.imread(os.path.join(PI_images_folder, img_id), cv2.IMREAD_GRAYSCALE)

    PI_tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = PI_fxfycxcy, tag_size=0.087)
    if len(PI_tags) == 0:
        continue

    # Here I select simply one tag for the test
    PI_tag = PI_tags[-1]
    Link_Register = register[PI_tag.tag_id]
    PI_tag_pose = np.concatenate(
                    [np.concatenate([np.array(PI_tag.pose_R), np.array(PI_tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
    # PI_Cam2World
    PI_registration[int(img_id.split(".")[0])] =  Link_Register.CameraPose \
                                                @ additional_rotation @ Link_Register.TagPose @ np.linalg.inv(PI_tag_pose)


#print(PI_registration)


In [51]:
IPhone_Registration = {}
GT_CameraPose = {}

for frame in os.listdir(iphone_frames_folder):
    if frame in track_frame.values():
        print("Passed")
        continue
    camera_param_file = frame.replace(".jpg", ".json")
    img = cv2.imread(os.path.join(iphone_frames_folder, frame), cv2.IMREAD_GRAYSCALE)

    img_height, img_width = img.shape
    with open(os.path.join(iphone_pose_folder, camera_param_file), 'r') as f:
        camera_param = json.load(f)

    intrinsics = np.array(camera_param["intrinsics"]).reshape(3, 3)
    Cam2World = np.array(camera_param["cameraPoseARFrame"]).reshape(4, 4)

    fxfycxcy= [intrinsics[0, 0], intrinsics[1, 1], intrinsics[0, 2], intrinsics[1, 2]]
    Iphone_tags = at_detector.detect(img, estimate_tag_pose=True, camera_params = fxfycxcy, tag_size=0.087)
    
    if len(Iphone_tags) == 0:
        continue
    Iphone_tag = Iphone_tags[0]
    
    
    Link_Register = register[Iphone_tag.tag_id]
    Iphone_tag_pose = np.concatenate(
                    [np.concatenate([np.array(Iphone_tag.pose_R), np.array(Iphone_tag.pose_t)], axis=1), np.array([[0, 0, 0, 1]])], axis=0)
    # PI_Cam2World
    IPhone_Registration[int(frame[6:11])] =  Link_Register.CameraPose @ additional_rotation \
                                                @ Link_Register.TagPose @ np.linalg.inv(Iphone_tag_pose)
    
    GT_CameraPose[int(frame[6:11])] = Cam2World @ additional_rotation


Passed


Error, more than one new minima found.


Passed


Error, more than one new minima found.


!!
Passed
Passed
!!
Passed


Error, more than one new minima found.


Passed


In [48]:
pos_error = []
rot_error = []
for gt_pose, est_pose in zip(GT_CameraPose.values(), IPhone_Registration.values()):
    pos_error.append(position_error(est_pose[:3, 3], gt_pose[:3, 3]))
    rot_error.append(rotation_error(est_pose[:3, :3], gt_pose[:3, :3]))


In [60]:
print(pos_error)
#print(rot_error)

[1.123390256719358, 1.159796056475863, 2.6365642349417984, 0.8676330072609346, 0.12122319913206958, 2.6190499889576633, 1.7096162336502954, 0.3884903758549655, 0.16209034585126567, 0.3179002427648617, 0.6552598802542235, 1.930755812379908, 1.0452871353041913, 0.9575329461967732, 0.618380025491959, 0.20044754530162462, 0.6714375587605254, 0.7307978960766717, 0.06025473639397546, 0.6297980737813117, 0.40835669267250607, 0.18995614177565642, 1.0755181084883196, 0.4019318206143533, 0.23646957514635533, 0.8760745025838655, 0.1145831061061343, 0.19998547469113334, 0.1430091059377007, 0.17195424301139348, 0.9874435095169267, 0.07167341558750885, 0.02271497865891587, 0.7218254377285533, 0.14983852744704973, 0.39066947748013187, 2.610648705736589, 0.8091163774747009]


In [61]:
print(rot_error)

[8.056352370011052, 9.879733784407383, 90.0723912368199, 7.4432661536743305, 4.581211794640577, 90.03090195805375, 19.63042742818509, 7.115298311393242, 4.557795233148411, 6.261001362693382, 10.455888527464257, 13.840148057494448, 8.647531289410244, 7.440157897567439, 11.343305343033022, 6.730065915634455, 7.477701993977995, 8.952275416436503, 1.385068335260358, 10.966423903957812, 7.369274789908186, 5.460402817513076, 7.000740773124104, 7.187249018109046, 7.0401046218068455, 8.128704388979362, 1.9489432525312607, 6.205918044249047, 3.292084845497263, 3.936132558008007, 7.7291021333897625, 3.113818435696691, 0.9753621536049437, 11.473184255713758, 4.727277382194814, 11.385697549881652, 90.0080436489519, 7.62859400344712]


In [52]:
mesh_textured = o3d.io.read_triangle_mesh(mesh_fullpath, True)

In [59]:
# Visualization of Camera Pose
visualization_registration(PI_registration, mesh_textured)


Output()